In [1]:
import pandas as pd
from config import *
from db_config import *
session,engine = connect_db()

import sqlalchemy
from sqlalchemy import create_engine,Column,Integer,String,ForeignKey,Table,Text,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship

from datetime import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob
from shutil import copyfile
import os

import matplotlib
font = {'size'   : 21}
matplotlib.rc('font', **font)

In [3]:
acceptable_status = ['received'] # List with all of the status that you want to pull from
rework = ['cloning_error','cloning_failure','trans_failure']
enzyme = 'BbsI'

def list_to_string(ls):
    string = ''
    for l in ls:
        string += "'{}',".format(l)
    return string[:-1]

def query_for_parts(status,enzyme,engine):
    query_parts = "SELECT parts.part_id,parts.status,fragments.fragment_name,plates.plate_id,wells.address,wells.volume,plates.id FROM parts\
            INNER JOIN part_frag ON parts.id = part_frag.part_id\
            INNER JOIN fragments ON part_frag.fragment_id = fragments.id\
            INNER JOIN wells ON fragments.id = wells.fragment_id\
            INNER JOIN plates on wells.plate_id = plates.id\
            WHERE parts.status IN ({})\
            AND parts.cloning_enzyme = '{}'".format(list_to_string(status),enzyme)

    return pd.read_sql_query(query_parts, con=engine)


to_build = query_for_parts(acceptable_status,enzyme,engine)
print(to_build.status.value_counts())
to_build

received    2924
Name: status, dtype: int64


,part_id,status,fragment_name,plate_id,address,volume,id
0,BBF10K_003118,received,BBF10K_003118_1,syn_plate071,B8,74.0,200
1,BBF10K_003119,received,BBF10K_003119_1,syn_plate071,B5,52.0,200
2,BBF10K_003119,received,BBF10K_003119_2,syn_plate071,C5,52.0,200
3,BBF10K_003120,received,BBF10K_003120_1,syn_plate071,A4,106.0,200
4,BBF10K_003121,received,BBF10K_003121_1,syn_plate071,A8,74.0,200
5,BBF10K_003122,received,BBF10K_003122_1,syn_plate071,E12,109.0,200
6,BBF10K_003123,received,BBF10K_003123_1,syn_plate071,G12,53.0,200
7,BBF10K_003123,received,BBF10K_003123_2,syn_plate071,H12,51.0,200
8,BBF10K_003124,received,BBF10K_003124_link_BBF10K_003226,syn_plate070,G5,54.0,199
9,BBF10K_003125,received,BBF10K_003125_1,syn_plate071,D1,81.0,200


In [5]:
def good(x):
    if x in used_plates:
        return 0
    else:
        return 1

used_plates = ['syn_plate022','syn_plate028']
to_build.id = to_build.plate_id.apply(good)
# print(to_build.address.value_counts())
# for plate in session.query(Plate).filter(Plate.plate_id.in_(used_plates)):
#     print(len(plate.wells))
# used_df = to_build[to_build.address == 0]
# print(used_df.fragment_name.value_counts())

In [6]:
print(to_build.id.value_counts())
grouped = to_build.groupby('part_id').filter(lambda x: len(x) == x['id'].sum())
print(grouped.id.value_counts())

# grouped.plate_id.unique().tolist()
# print(len(grouped))

1    2734
0     190
Name: id, dtype: int64
1    2725
Name: id, dtype: int64


In [18]:
sort_group = grouped.sort_values('plate_id')
sub = sort_group.part_id.unique().tolist()[:45]
if len(sub) < 96:
    new = pd.concat([sort_group,query_for_parts(rework,enzyme,engine)])
new = new[:96]
new
sub_df = sort_group[sort_group.part_id.isin(sub)]
# print(sub_df.plate_id.value_counts())
# print(len(sub_df.part_id.unique().tolist()))
sub_group = sub_df.groupby('part_id').agg(len)
print(sub_group.status.value_counts())
print(sub_df.groupby('part_id').agg(len).status.value_counts())

1    55
2    38
3     2
4     1
Name: status, dtype: int64
1    55
2    38
3     2
4     1
Name: status, dtype: int64


In [19]:
import ot_functions as ot
build_parts = sub_df.part_id.unique().tolist()
build_plan = pd.DataFrame({
    'Gene': build_parts,
    'Destination': ot.well_addresses()[:len(build_parts)]
})
build_plan

,Destination,Gene
0,A1,BBF10K_002389
1,B1,BBF10K_002871
2,C1,BBF10K_001344
3,D1,BBF10K_001610
4,E1,BBF10K_002503
5,F1,BBF10K_001786
6,G1,BBF10K_001467
7,H1,BBF10K_002557
8,A2,BBF10K_001333
9,B2,BBF10K_002552


In [ ]:
current_ids = pd.read_sql_query("SELECT plates.id,plates.plate_name,plates.plate_id FROM plates WHERE plates.plate_id IN ('syn_plate032', 'syn_plate033', 'syn_plate034', 'syn_plate038', 'syn_plate039', 'syn_plate040', 'syn_plate041', 'syn_plate042', 'syn_plate043', 'syn_plate044', 'syn_plate047', 'syn_plate048', 'syn_plate049', 'syn_plate052', 'syn_plate053', 'syn_plate054', 'syn_plate055', 'syn_plate056', 'syn_plate057', 'syn_plate058')", con=engine)
# current_ids = current_ids[current_ids.plate_name != 'pSHPs0521B555236MU']
# current_ids.plate_id = current_ids.plate_id.apply(lambda x: int(x[-3:]))
# all_ids = range(max(current_ids.plate_id.tolist()))
# missing_ids = [x for x in all_ids if x not in current_ids.plate_id.tolist()]
# print(missing_ids)
# if missing_ids == []:
#     next_id = max(current_ids.plate_id.tolist()) + 1
# else:
#     next_id = missing_ids[0]
# print(next_id)
print(current_ids.id.tolist())

In [ ]:
query_plates = "SELECT * FROM plates"

df_plates = pd.read_sql_query(query_plates, con=engine)
df_syn = df_plates[df_plates.plate_type == 'syn_plate']
# df_syn = df_plates[df_plates.resuspended == 'resuspended']
df_syn = df_syn.sort_values('id')
df_syn = df_syn.reset_index()
print(len(df_syn.plate_name.unique()))
print(len(df_syn))
df_syn
used = []
used_id = []
dup = []
for plate,plate_id in zip(df_syn.plate_name.tolist(),df_syn.plate_id.tolist()):
    current = session.query(Plate).filter(Plate.plate_id == plate_id).one()
    print(len(current.wells))
    if plate in used:
        print(plate,plate_id,"Already used",df_syn[df_syn.plate_name == plate].plate_id.tolist())
        dup.append(plate_id)
    else:
        print(plate,plate_id)
        used.append(plate)
        used_id.append(plate_id)
print(dup)
# df_syn = df_syn.reset_index()
# df_syn.plate_id = df_syn.level_0.apply(lambda x: 'syn_plate{}'.format(str(x).zfill(3)))
# df_syn = df_syn[['plate_name','plate_id']]
# df_syn.to_csv('./plate_id_dictionary')
# plate_dict = dict(zip(df_syn.id,df_syn.plate_id))
# plate_dict
# for plate in session.query(Plate).filter(Plate.plate_type == 'syn_plate'):
#     plate.plate_id = plate_dict[plate.id]
#     print(plate.id,plate.plate_id)
# session.commit()

In [ ]:
for plate in used_id:
    obj = session.query(Plate).filter(Plate.plate_id == plate).one()
    print(obj.plate_name)
    for well in obj.wells:
        if well.volume != None:
            print(well.address,well.volume)

In [ ]:
query_plates = "SELECT * FROM plates"

df_plates = pd.read_sql_query(query_plates, con=engine)
df_syn = df_plates[df_plates.plate_type == 'syn_plate']
# df_syn = df_plates[df_plates.resuspended == 'resuspended']
df_syn = df_syn.sort_values('id')
df_syn = df_syn.reset_index()
print(len(df_syn.plate_name.unique()))
print(len(df_syn))
df_syn

In [ ]:
build = session.query(Build).filter(Build.build_name == 'build011').one()
print(build.build_name,build.status)
build.build_name = 'build011'
# session.commit()
print(build.build_name)

In [ ]:
last_id = session.query(Plate).filter(Plate.plate_type == 'syn_plate')\
            .filter(Plate.resuspended == 'resuspended').order_by(Plate.plate_id)[-1]
print(last_id.plate_id)
print('syn_plate'+str(int(last_id.plate_id[-3:])+1).zfill(3))
for plate in session.query(Plate).filter(Plate.plate_type == 'syn_plate')\
            .filter(Plate.resuspended == 'not_resuspended'):
    print(plate.plate_name)
    plate.add_plate_id(session)
    print(plate.plate_id)

In [ ]:
print(datetime.now(),'Began run')

query_outcomes = "SELECT parts.part_id,parts.status,wells.seq_outcome,wells.plate_type,builds.build_name,wells.misplaced FROM parts \
        INNER JOIN wells ON parts.id = wells.part_id\
        INNER JOIN plates ON wells.plate_id = plates.id\
        INNER JOIN builds ON plates.build_id = builds.id"

query_frag = "SELECT parts.part_id,fragments.fragment_name,twist_orders.sub_name FROM parts\
        INNER JOIN part_frag ON parts.id = part_frag.part_id\
        INNER JOIN fragments ON part_frag.fragment_id = fragments.id\
        INNER JOIN frag_order ON fragments.id = frag_order.frag_id\
        INNER JOIN twist_orders ON twist_orders.id = frag_order.twist_order_id"

query_parts = "SELECT * FROM parts"

df_frag = pd.read_sql_query(query_frag, con=engine)
frags = df_frag.groupby('part_id')['fragment_name'].agg(len)
frags.name = 'Count'
frags = pd.DataFrame(frags).reset_index()
frags_dict = dict(zip(frags.part_id.tolist(),frags.Count.tolist()))
subs_dict = dict(zip(df_frag.part_id.tolist(),df_frag.sub_name.tolist()))

author_dict = []
for file in sorted(glob.glob('../data/*/*.json')):
    with open(file,"r") as json_file:
        data = json.load(json_file)
    author_dict.append([data['gene_id'],data['author']['name']])
author_dict = dict(author_dict)
# print(author_dict)
# input()
    

print(datetime.now(),'Finished frags')

def multiple(x):
    if len(x) == 1:
        x.append('N/A')
    return x

def find_outcome(x):
    if x in df_out_dict.keys():
        return df_out_dict[x]
    else:
        return ['N/A','N/A']
    
def find_build(x):
    if x in df_build_dict.keys():
        return df_build_dict[x]
    else:
        return ['N/A','N/A']
    
def simplify_outcome(x):
    if "mutation" in x:
        return 'cloning_mutation'
    elif "bad" in x:
        return 'sequence_failure'
#     elif x == 'cloning_error':
#         return 'cloning_failure'
    else:
        return x
    
def find_author(x):
    return author_dict[x]

df_res = pd.read_sql_query(query_outcomes, con=engine)
df_res = df_res[df_res.plate_type == 'seq_plate']

df_out = df_res.groupby('part_id')['seq_outcome'].apply(list)
df_out.name = 'Outcomes'
df_out = pd.DataFrame(df_out).reset_index()
df_out.Outcomes = df_out.Outcomes.apply(multiple)
df_out_dict = dict(zip(df_out.part_id.tolist(),df_out.Outcomes.tolist()))

df_build = df_res.groupby('part_id')['build_name'].apply(list)
df_build.name = 'Builds'
df_build = pd.DataFrame(df_build).reset_index()
df_build.Builds = df_build.Builds.apply(multiple)
df_build_dict = dict(zip(df_build.part_id.tolist(),df_build.Builds.tolist()))
print(datetime.now(),'Finished outcomes')

df_parts = pd.read_sql_query(query_parts, con=engine)
print('finished part query')

df_parts = df_parts[df_parts.part_id != 'BBF10K_000745']

df_parts['Fragments'] = df_parts.part_id.apply(lambda x: frags_dict[x])
df_parts['Submission'] = df_parts.part_id.apply(lambda x: subs_dict[x])
df_parts['Order_number'] = df_parts.Submission.apply(lambda name: int(name[-3:]))
df_parts['Outcomes'] = df_parts.part_id.apply(find_outcome)
df_parts['Builds'] = df_parts.part_id.apply(find_build)
print('finished outcome and builds')
df_parts['Attempt_1_Outcome'] = df_parts.Outcomes.apply(lambda x: x[0])
df_parts['Attempt_1_Outcome_G'] = df_parts.Attempt_1_Outcome.apply(simplify_outcome)
df_parts['Attempt_1_Build'] = df_parts.Builds.apply(lambda x: x[0])
df_parts['Attempt_2_Outcome'] = df_parts.Outcomes.apply(lambda x: x[1])
df_parts['Attempt_2_Outcome_G'] = df_parts.Attempt_2_Outcome.apply(simplify_outcome)
df_parts['Attempt_2_Build'] = df_parts.Builds.apply(lambda x: x[1])
df_parts['Length'] = df_parts.seq.apply(len)
df_parts['Author'] = df_parts.part_id.apply(find_author)
print(datetime.now(),'Finished building dataframe')
df_parts

In [ ]:
df_good = df_parts[df_parts.status == 'sequence_confirmed']
df_good = df_good.sort_values('Length')
df_good

In [ ]:
data_b = df_parts[df_parts.cloning_enzyme == 'BbsI']
data_b

data_good = data_b[data_b.status == 'sequence_confirmed']
print(len(data_good))
print(data_good.Length.sum())
data_good = data_good.sort_values('Length')
data_good

In [ ]:
data_wol = data_b[data_b.Author == 'Grant Hartzog']
# data_wol = data_wol[data_wol.status == 'sequence_confirmed']
data_wol


In [ ]:
build = session.query(Build).filter(Build.build_name == '').one()
# for build in builds:
#     print(build.build_name)
print(build.plates[0].wells[0].parts.part_id)

In [ ]:
query_seq = "SELECT parts.part_name,parts.seq FROM parts"

df_seq = pd.read_sql_query(query_seq, con=engine)
df_seq.to_csv('./freegenes_sequences.csv')

In [ ]:
## BBREAKDOWN OF OUTCOMES BY NUMBER OF FRAGMENTS

data_b = df_parts[df_parts.cloning_enzyme == 'BbsI']

# data_fail = data_b[data_b.status != 'sequence_confirmed']
data_att = data_b[data_b.status != 'ordered']

frag_norm = (data_att.groupby('Fragments')['status'].value_counts() / data_att.groupby('Fragments')['status'].agg(len))
frag_norm.name = 'Percent'
norm_frag_df = pd.DataFrame(frag_norm).reset_index()
norm_frag_df.Percent = norm_frag_df.Percent * 100

dims = (16, 6)

fig1, ax1 = plt.subplots(figsize=dims)
g = sns.barplot(ax=ax1,data=norm_frag_df, x='Fragments', y='Percent', hue='status',palette="Spectral")
g.set_yscale('log')
plt.title("Normalized Stat Percentage vs. Number of Fragments")
plt.show()

total_frag_raw = (data_att.groupby('Fragments')['status'].value_counts())
total_frag_raw.name = 'Count'
total_raw = pd.DataFrame(total_frag_raw).reset_index()

fig2, ax2 = plt.subplots(figsize=dims)
t = sns.barplot(ax=ax2,data=total_raw, x='Fragments', y='Count', hue='status',palette='Spectral')
t.set_yscale('log')
plt.title("Raw Status Count vs. Number of Fragments")
plt.show()

print(datetime.now(),'Finished outcome vs. fragments\n')

data_att.groupby(data_att.status).Fragments.describe()


In [ ]:
## CLONING FAILURE VERSUS LENGTH

data_fail = data_att[data_att.status != 'sequence_confirmed']
data_fail = data_fail[data_fail.status != 'abandoned']


fig3, ax3 = plt.subplots()

data_fail.groupby(data_fail.status).Length.plot.hist(ax=ax3,alpha=0.5,legend=True,figsize=(16,6),bins=10)
data_fail.groupby(data_fail.status).Length.describe()

plt.xlabel('Seqence Length (bp)')
plt.ylabel('Counts')
plt.title('Cloning Failure vs. Sequence Length')

# fig3.savefig('../docs/Overall/raw_length.png')




In [ ]:
## SYNTHESIS FAILURES PLOTTED AGAINST SEQUENCE LENGTH

data_order = data_b[data_b.status == 'abandoned']
fig4, ax4 = plt.subplots()

ax4.set_xticks([1000,2000,3000,4000,5000])

data_order.groupby(data_order.status).Length.plot.hist(ax=ax4,alpha=0.5,legend=True,figsize=(16,6),bins=10)

plt.xlabel('Seqence Length (bp)')
plt.ylabel('Counts')
plt.title('Synthesis Abandonment Versus Sequence Length')
plt.show()
# fig4.savefig('../docs/Overall/syn_fail.png')

data_order.Length.describe()


In [ ]:
## SECOND ATTEMPT BREAKDOWN

# Elimination of irrelevant information for the plots
data_att = data_b[data_b.status != 'ordered']
data_att = data_att[data_att.status != 'received']
data_att = data_att[data_att.Attempt_1_Outcome_G != 'sequence_confirmed']
data_attempt = data_att[data_att.Attempt_2_Outcome_G != 'N/A']

# Normalizing the data by dividing the counts of each 2nd outcome by the total counts of each group of 1st outcomes
attempts_norm = (data_attempt.groupby('Attempt_1_Outcome_G')['Attempt_2_Outcome_G'].value_counts() / data_attempt.groupby('Attempt_1_Outcome_G')['Attempt_2_Outcome_G'].agg(len))
attempts_norm.name = 'Percent'
attempts_norm = pd.DataFrame(attempts_norm).reset_index()
attempts_norm.Percent = attempts_norm.Percent * 100

# Plot normalized data
dims = (16, 6)
fig1, ax = plt.subplots(figsize=dims)
p = sns.barplot(data=attempts_norm, x='Attempt_1_Outcome_G', y='Percent', hue='Attempt_2_Outcome_G',palette="Spectral").set_title("Normalized 2nd Attempt Outcomes")
plt.xlabel("1st Attempt Outcome")
plt.show()
# fig1.savefig('../docs/Overall/norm_attempt.png')

# Caluclate the raw counts
attempts_raw = (data_attempt.groupby('Attempt_1_Outcome_G')['Attempt_2_Outcome_G'].value_counts())
attempts_raw.name = 'Count'
attempts_raw = pd.DataFrame(attempts_raw).reset_index()

# Plot the raw counts
fig2, ax = plt.subplots(figsize=dims)
g = sns.barplot(data=attempts_raw, x='Attempt_1_Outcome_G', y='Count', hue='Attempt_2_Outcome_G',palette="Spectral").set_title("2nd Attempt Outcomes")
plt.xlabel("1st Attempt Outcome")
plt.show()
# fig2.savefig('../docs/Overall/raw_attempt.png')


In [ ]:
## BREAKING DOWN CLONING OUTCOMES BY ORDER NUMBER

# Normalizing the data by dividing the counts of each 2nd outcome by the total counts of each group of 1st outcomes
order_norm = (data_b.groupby('Order_number')['status'].value_counts() / data_b.groupby('Order_number')['status'].agg(len))
order_norm.name = 'Percent'
order_norm = pd.DataFrame(order_norm).reset_index()
order_norm.Percent = order_norm.Percent * 100

# Plot normalized data
dims = (16, 6)
fig1, ax = plt.subplots(figsize=dims)
p = sns.barplot(data=order_norm, x='Order_number', y='Percent', hue='status',palette="Spectral").set_title("Normalized Outcomes by Order")
plt.xlabel("Order Number")
plt.show()
# fig1.savefig('../docs/Overall/norm_order.png')

# Caluclate the raw counts
order_raw = (data_b.groupby('Order_number')['status'].value_counts())
order_raw.name = 'Count'
order_raw = pd.DataFrame(order_raw).reset_index()

# Plot the raw counts
fig2, ax = plt.subplots(figsize=dims)
g = sns.barplot(data=order_raw, x='Order_number', y='Count', hue='status',palette="Spectral")

g.set_yscale('log')
plt.title("Outcome Counts by Order")
plt.xlabel("Order Number")
plt.show()
# fig2.savefig('../docs/Overall/raw_order.png')

In [ ]:
## BUILD ATTEMPT BREAKDOWN

df_int = df_res
df_int['Outcome'] = df_int.seq_outcome.apply(simplify_outcome)

# Normalizing the data by dividing the counts of each 2nd outcome by the total counts of each group of 1st outcomes
build_norm = (df_int.groupby('build_name')['Outcome'].value_counts() / df_int.groupby('build_name')['Outcome'].agg(len))
build_norm.name = 'Percent'
build_norm = pd.DataFrame(build_norm).reset_index()
build_norm.Percent = build_norm.Percent * 100

# Plot normalized data
dims = (16, 6)
fig1, ax = plt.subplots(figsize=dims)
p = sns.barplot(data=build_norm, x='build_name', y='Percent', hue='Outcome',palette="Spectral").set_title("Normalized Outcomes by Build")
plt.xlabel("Build Number")
plt.show()
# fig1.savefig('../docs/Overall/norm_build.png')

# Caluclate the raw counts
build_raw = (df_int.groupby('build_name')['status'].value_counts())
build_raw.name = 'Count'
build_raw = pd.DataFrame(build_raw).reset_index()

# Plot the raw counts
fig2, ax = plt.subplots(figsize=dims)
g = sns.barplot(data=build_raw, x='build_name', y='Count', hue='status',palette="Spectral")

# g.set_yscale('log')
plt.title("Outcome Counts by Build")
plt.xlabel("Build Number")
plt.show()
# fig2.savefig('../docs/Overall/raw_build.png')


for i,build in build_norm.groupby('build_name'):
    print(build,'\n')

In [ ]:
## GENERATE SANKEY DIAGRAMS

def add_branch(source,target,amount,nodes,links):
    if amount == 0:
        return nodes,links
    nodes += [source,target]
    links.append([source,target,amount])
    return nodes,links

def gen_sankey(nodes,links,sankey={'nodes':[],'links':[]}):
    node_dict = dict([[y,x] for x,y in enumerate(pd.Series(nodes).unique())])
    print(node_dict)
    for name in node_dict.keys():
        sankey['nodes'].append({'name' : name})
    for source,target,value in links:
        sankey['links'].append({
            "source":node_dict[source],
            "target":node_dict[target],
            "value":value
        })
    return sankey

desired_dfs = [df[1] for df in data_b.groupby('Order_number')]
df_names = ["Order_"+str(num+1).zfill(3) for num in range(len(data_b.groupby('Order_number')))]
print(df_names)

desired_dfs.append(data_b)
df_names.append('Overall')

for name,df in zip(df_names,desired_dfs):
        
    sankey = {}

    nodes = ['Total_ordered']
    links = []
    sankey['nodes'] = []
    sankey['links'] = []
    
    total = len(df)
    
    if name != 'Overall':
        authors = pd.DataFrame(df.Author.value_counts())
        author_names = [str(author) for author in authors.index.tolist()]
        author_counts = [int(count) for count in authors.Author.tolist()]

        for author,count in zip(author_names,author_counts):
            nodes,links = add_branch(author,'Total_ordered',count,nodes,links)
    else:
        # Orders
        orders = pd.DataFrame(df.Order_number.value_counts())
        order_names = ['Order_'+str(order).zfill(3) for order in orders.index.tolist()]
        order_counts = [int(count) for count in orders.Order_number.tolist()]

        for order,count in zip(order_names,order_counts):
            nodes,links = add_branch(order,'Total_ordered',count,nodes,links)

    # Synthesizing
    syn = len(df[df.status == 'ordered'])
    nodes,links = add_branch('Total_ordered','Synthesizing',syn,nodes,links)

        
    # Abandoned
    abandoned = len(df[df.status == 'abandoned'])
    nodes,links = add_branch('Total_ordered','Abandoned',abandoned,nodes,links)

    # Received
    received = total-abandoned-syn
    nodes,links = add_branch('Total_ordered','Received',received,nodes,links)

    # Attempted
    not_attempted = len(df[df.status == 'received'])
    nodes,links = add_branch('Received','Not_attempted',not_attempted,nodes,links)
    attempted = received-not_attempted
    nodes,links = add_branch('Received','Attempted',attempted,nodes,links)

    # Outcomes 
    data_att = df[df.status != 'ordered']
    data_att = data_att[data_att.status != 'abandoned']
    outcomes = pd.DataFrame(data_att.status.value_counts())
    out = outcomes.index.tolist()
    count = outcomes.status.tolist()
    for out,count in zip(out,count):
        print(out,count)
        nodes,links = add_branch('Attempted',out,count,nodes,links)

    sankey = gen_sankey(nodes,links,sankey=sankey)
    
    path = '{}/docs/{}'.format(BASE_PATH,name)
    if os.path.exists(path):
        print("Directory for {} already exists".format(name))
    else:
        # Generates a new directory with the ID# as its name
        os.makedirs(path)
        print("Making directory for {}".format(name))    

    with open("{}/sankey.json".format(path),"w+") as json_file:
        json.dump(sankey,json_file,indent=2)
        
    copyfile('../docs/sankey/sankey.html','{}/sankey.html'.format(path))
        
    date = str(datetime.now()).split(" ")[0]
    if name == 'Overall':
        with open('../docs/sankey/index.md','r') as md_file:
            data = md_file.read()
            print(data)
            data = data.replace('[DATE]',date)
            print(data)
        with open('{}/index.md'.format(path),'w') as md_file:
            md_file.write(data)
    else:
        with open('../docs/sankey/order.md','r') as md_file:
            data = md_file.read()
            print(data)
            data = data.replace('[ORDER]',name)
            data = data.replace('[DATE]',date)
            print(data)
        with open('{}/index.md'.format(path),'w') as md_file:
            md_file.write(data)

        
    print("Check sankey")
    input()

In [ ]:
# with open('../docs/sankey/index.md','r') as md_file:
#     data = md_file.read()
#     print(data)
#     data = data.replace('[INSERT LINK]','inserted')
#     print(data)
# with open('../docs/sankey/index.md','w') as md_file:
#     md_file.write(data)

In [ ]:
# orders = ['submission009','submission010','submission011']
# aband = df_parts[df_parts.apply(lambda row: row.Submission not in orders, axis=1)]
# to_aband = aband[aband.status == 'ordered']
# print(len(to_aband))
# to_abandon = to_aband.part_id.tolist()

# for part in to_abandon:
#     current = session.query(Part).filter(Part.part_id == part).one()
#     current.status = 'abandoned'
    
# session.commit()

In [ ]:
fail2 = df_parts[df_parts.Attempt_1_Outcome_G == 'cloning_failure']
fail2 = fail2[fail2.Attempt_2_Outcome_G == 'cloning_failure']
fail2_limit = fail2[['part_id','part_name','Length','Fragments','part_type']]
fail2_limit

string = '| Part ID | Gene Name | Sequence Length | # of Fragments | Part Type |\n'
string += '| ------------- | ------------- | :-------------: | :-------------: | ------------- |\n'
for i,row in fail2_limit.iterrows():
    string += '| {} | {} | {} | {} | {} |\n'.format(row.part_id,row.part_name,row.Length,row.Fragments,row.part_type)
    
total_ordered_bp = df_parts.Length.sum()
total_ordered_g = len(df_parts)

rec = ['ordered','abandoned']
total_received = df_parts[df_parts.apply(lambda row: row.status not in rec, axis=1)]
total_received_bp = total_received.Length.sum()
total_received_g = len(total_received)

total_built = df_parts[df_parts.status == 'sequence_confirmed']
total_built_bp = total_built.Length.sum()
total_built_g = len(total_built)

rework = ['build008','build010']
normal_builds = df_parts[df_parts.apply(lambda row: row.Attempt_1_Build not in rework, axis=1)]
total_attempted = normal_builds[normal_builds.Attempt_1_Outcome_G != 'N/A']
total_built_first = normal_builds[normal_builds.Attempt_1_Outcome_G == 'sequence_confirmed']
total_attempted_g = len(total_attempted)
total_built_first_g = len(total_built_first)
success_rate = round((total_built_first_g / total_attempted_g)*100)


overall = '| Amount Ordered | Amount Received  | Amount Built | Cloning Success Rate |\n'
overall += '| :-------------: | :-------------: | :-------------: | :-------------: |\n'
overall += '| {}bp / {}genes | {}bp / {}genes | {}bp / {}genes | {}% |'.format(total_ordered_bp,total_ordered_g,total_received_bp,total_received_g,total_built_bp,total_built_g,success_rate)
    
with open('../docs/Overall/index.md','r') as md_file:
    data = md_file.read()
    print(data)
    data = data.replace('[Failure Table]',string)
    data = data.replace('[General Table]',overall)
    print(data)
with open('../docs/Overall/index.md'.format(path),'w') as md_file:
    md_file.write(data)


In [ ]:
total_ordered_bp = df_parts.Length.sum()
total_ordered_g = len(df_parts)

rec = ['ordered','abandoned']
total_received = df_parts[df_parts.apply(lambda row: row.status not in rec, axis=1)]
total_received_bp = total_received.Length.sum()
total_received_g = len(total_received)

total_built = df_parts[df_parts.status == 'sequence_confirmed']
total_built_bp = total_built.Length.sum()
total_built_g = len(total_built)

rework = ['build008','build010']
normal_builds = df_parts[df_parts.apply(lambda row: row.Attempt_1_Build not in rework, axis=1)]
total_attempted = normal_builds[normal_builds.Attempt_1_Outcome_G != 'N/A']
total_built_first = normal_builds[normal_builds.Attempt_1_Outcome_G == 'sequence_confirmed']
total_attempted_g = len(total_attempted)
total_built_first_g = len(total_built_first)
success_rate = round((total_built_first_g / total_attempted_g)*100)


overall = '| Amount Ordered | Amount Received  | Amount Built | Cloning Success Rate |\n'
overall += '| :-------------: | :-------------: | :-------------: | :-------------: |\n'
overall += '| {} bp / {} genes | {} bp / {} genes | {} bp / {} genes | {}% |'.format(total_ordered_bp,total_ordered_g,total_received_bp,total_received_g,total_built_bp,total_built_g,success_rate)

print(success_rate)
print(overall)